In [1]:
import kaggle

In [4]:
# !kaggle competitions download -c house-prices-advanced-regression-techniques


  0%|          | 0.00/199k [00:00<?, ?B/s]
100%|##########| 199k/199k [00:00<00:00, 8.06MB/s]


In [5]:
# !unzip house-prices-advanced-regression-techniques.zip

Archive:  house-prices-advanced-regression-techniques.zip
  inflating: data_description.txt    
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [14]:
import pandas as pd
import numpy as np 

### Feature Engineering의 목표
- 데이터를 문제에 맞게 바꾸는 것
    - 예를 들면 `겉보기 온도`라는 개념은 습도, 기온, 풍속 등 직접 측정할 수 있는 것들에서 옴
    - 따라서 이것 또한 Feature Engineering이라고 할 수 있다
   

Feature Engineering은 이런 것들을 위해 쓰인다 : 
- 모델의 예측 성능 향상
- 데이터나 계산의 수 줄이기
- 결과의 해석 가능성 향상

유용한 Feature는 "모델이 배울 수 있는" 타겟과의 관계를 가져야 한다.
- 선형 모델을 예로 들면, 선형적인 관계만을 학습할 수 있기 때문에 특성 공학을 이용해 여러 Feature들을 타겟값과 선형성을 갖도록 바꿔야 한다.  
    
핵심은, **특성에 적용하는 변환이 모델의 일부가 된다**는 것이다.
- 어떤 y 피쳐가 X 피쳐와 제곱 관계를 나타낸다고 했을 때 선형회귀를 적용한다면 올바른 회귀선이 나오지 않을 것임
- 이 떄 X 피쳐를 제곱한 뒤 선형회귀를 적용하면 훨씬 나은 플롯을 보여준다.

## 예시

In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

In [19]:
COLUMNS = ['Cement', 'BlastFurnaceSlag', 'FlyAsh', 'Water', 'SuperPlasticizer', 'CoarseAggregate', 'FineAggregate', 'Age', 'CompressiveStrength']
df = pd.read_excel('Concrete_Data.xls', names = COLUMNS)
df.head()

,Cement,BlastFurnaceSlag,FlyAsh,Water,SuperPlasticizer,CoarseAggregate,FineAggregate,Age,CompressiveStrength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.986111
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.887366
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.269535
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.052780
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.296075


### 1. 베이스라인 모델 : 아무 특성도 추가하지 않음

In [22]:
X = df.copy()
y = X.pop('CompressiveStrength')

baseline = RandomForestRegressor(criterion = 'mae', random_state = 0)
baseline_score = cross_val_score(baseline, X, y, cv = 5, scoring = 'neg_mean_absolute_error')
baseline_score = -1 * baseline_score.mean()

print(f"MAE Baseline Score : {baseline_score:.4}")

MAE Baseline Score : 8.397


## 2. "비율" 특성 추가
- 요리할 때 레시피에서 재료의 `비율`은 결과를 결정하는 주요 지표 중 하나임
- 이런 것처럼 특성의 `Ratio`도 그러한 지표로 활용할 수 있음

In [23]:
X = df.copy()
y = X.pop('CompressiveStrength')

# 합성 지표 만들기
X['FCRatio'] = X['FineAggregate'] / X['CoarseAggregate']
X['AggCmtRatio'] = (X['CoarseAggregate'] + X['FineAggregate']) / X['Cement']
X['WtrCmtRatio'] = X['Water'] / X['Cement']

# 스코어
model = RandomForestRegressor(criterion = 'mae', random_state = 0)
model_score = cross_val_score(baseline, X, y, cv = 5, scoring = 'neg_mean_absolute_error')
model_score = -1 * model_score.mean()

print(f"MAE Score with Ratio Features: {model_score:.4}")

MAE Baseline Score : 8.01
